In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lsst.rsp import get_tap_service
import query as q

In [ ]:
service = get_tap_service("ssotap")
assert service is not None
results = service.search("SELECT column_name, datatype, description, "
                         "unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp03_catalogs_10yr.DiaSource'")
results.to_table().to_pandas()

In [ ]:
results['description'][9]

### Query testing

Object type given, no join

Object type given, join

Object type not given, no join

Object type not given, join

#### Object type given, no join

In [ ]:
query, object_type = q.make_query_general(object_type = "JFC")
print(query)
print(object_type)

query, object_type = q.make_query_general(object_type = "NEO")
print(query)
print(object_type)

query, object_type = q.make_query_general(object_type = "TNO")
print(query)
print(object_type)

#### Object type given, join

In [ ]:
query, object_type = q.make_query_general(object_type = "JFC", join = 'Diasource')
print(query)
print(object_type)

query, object_type = q.make_query_general(object_type = "NEO", join = 'Diasource')
print(query)
print(object_type)

query, object_type = q.make_query_general(object_type = "Ntrojan", join = 'Diasource')
print(query)
print(object_type)

#### Params given, join

In [ ]:
query, object_type = q.make_query_general(t_cutoff_min = 2.0, t_cutoff = 3.0, join = 'Diasource')
print(query)
print(object_type)

query, object_type = q.make_query_general(a_cutoff_min = 30.1, a_cutoff = 50)
print(query)
print(object_type)

In [ ]:
query, object_type = q.make_query_general(object_type = "JFC", join = 'Diasource')
print(query)
print(object_type)

query, object_type = q.make_query_general(t_cutoff_min = 2.0, t_cutoff = 3.0, join = 'Diasource')
print(query)
print(object_type) 